In [ ]:
!python3 -V

Python 3.8.15


In [ ]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install fastai==2.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.2 MB/s eta 0:04:21tcmalloc: large alloc 1147494400 bytes == 0x39118000 @  0x7fd2405c5615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:27tcmalloc: large alloc 1434370048 bytes == 0x7d76e000 @  0x7fd2405c5615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import gc

In [ ]:
print(torch. __version__)

1.7.1+cu110


In [ ]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/YelpDataset/dataframes

AutoDF.pkl		     HealthMedicalDF.pkl  restaurantDF.pkl
beautyandSpaDF.pkl	     HomeServicesDF.pkl   shoppingDF.pkl
EventPlanningServicesDF.pkl  LocalServicesDF.pkl
foodDF.pkl		     NightlifeDF.pkl


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/dataframes/restaurantDF.pkl /content/

In [ ]:
import pandas as pd
df = pd.read_pickle('restaurantDF.pkl')

In [ ]:
df = df.rename(columns={'text': 'reviewText'})
df = df.rename(columns={'stars': 'overall'})
df = df.rename(columns={'categories': 'category'})

In [ ]:
df = df[['business_id','name','city','state','overall','category','reviewText']]

In [ ]:
df.head(1)

,business_id,name,city,state,overall,category,reviewText
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,4.0,Restaurants,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area."


In [ ]:
df['sentenceLength'] = df['reviewText'].str.count(' ') + 1

In [ ]:
df.shape

(3773770, 8)

In [ ]:
df.head(2)

,business_id,name,city,state,overall,category,reviewText,sentenceLength
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,4.0,Restaurants,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area.",70
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,4.0,Restaurants,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before ...",116


In [ ]:
sample_df = df.sample(frac=0.05, random_state=2021).dropna().reset_index()

In [ ]:
sample_df.shape

(188688, 9)

In [ ]:
del df
gc.collect()

175

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [ ]:
!ls /content/drive/MyDrive/YelpDataset/models

gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Food.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping.pth


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/models/gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants.pth /content/

In [ ]:
learn.load("gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
"""N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 10
BINS = list(range(10,15))"""
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 2000
BINS = list(range(10,351))

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
generated_df_rows = []
for i in range(len(BINS)-1):
    sample_row_df = sample_df[(sample_df["sentenceLength"]>=BINS[i]) & (sample_df["sentenceLength"]<BINS[i+1])]
    bin_proportion = (sample_row_df.shape[0])/float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int((bin_proportion)*N_REVIEWS)+1
    try:
        or_df, cg_df =  train_test_split(sample_row_df, test_size=0.5, random_state=2021)
    except Exception as e:
        print(e)
        continue
    sample_reviews_df = cg_df.sample(n=n_reviews_to_generate,random_state=2021)
    or_sample_reviews_df = or_df.sample(n=n_reviews_to_generate,random_state=2021)
    if BINS[i] in range(10,21):
        max_len = 20
    else:
        max_len = BINS[i]
    for index, row in sample_reviews_df.iterrows():
        text = row["reviewText"]
        prompt = (" ").join(text.strip().split(" ")[:N_INITIAL_WORDS])
        prompt_ids = tokenizer.encode(prompt)
        inp = tensor(prompt_ids)[None].cuda()
        # set seed to reproduce results
        random_seed(seed_value=2021, use_cuda=True)
        preds = learn.model.generate(inp, max_length=max_len, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
        decoded_preds = tokenizer.decode(preds[0],skip_special_tokens=True)
        generated_df_rows.append([row["category"],row["overall"],decoded_preds,"CG"])
    for index, row in or_sample_reviews_df.iterrows():
        generated_df_rows.append([row["category"],row["overall"],row["reviewText"],"OR"])
#     break
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "rating", "text", "label"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
generated_df.shape

(4248, 4)

In [ ]:
generated_df.head()

,category,rating,text,label
0,Restaurants,4.0,love this place. great atmosphere. the food is good. my only complaint is the,CG
1,Restaurants,4.0,Best pizza in Sherwood Park. We have been going here for years and have always enjoyed the food,CG
2,Restaurants,4.0,"Love it! Good beans, better rice, amazing enchiladas, fabulous salsa!",OR
3,Restaurants,4.5,Little hipster and crowded for me but yummy chocolate croissants.,OR
4,Restaurants,3.5,terrible servile. not worthy of a second visit.We decided to try this place for the,CG


In [ ]:
def post_process(str, k=4):
    if not str.endswith("."):
        full_stop_split_str = str.split(".")
        clean_full_stop_split_str = [s.strip() for s in full_stop_split_str if s.strip()]
        if len(clean_full_stop_split_str) > 1:
            last_sentence = clean_full_stop_split_str[-1]
        else:
            return str
        last_sentence_len = len(last_sentence.split(" "))
        if last_sentence_len > k:
            return str
        return (". ").join(clean_full_stop_split_str[:-1]).strip()+(".")
    return str

In [ ]:
generated_df['text_'] = generated_df['text'].apply(lambda x: post_process(x))

In [ ]:
generated_df.head()

,category,rating,text,label,text_
0,Restaurants,4.0,love this place. great atmosphere. the food is good. my only complaint is the,CG,love this place. great atmosphere. the food is good. my only complaint is the
1,Restaurants,4.0,Best pizza in Sherwood Park. We have been going here for years and have always enjoyed the food,CG,Best pizza in Sherwood Park. We have been going here for years and have always enjoyed the food
2,Restaurants,4.0,"Love it! Good beans, better rice, amazing enchiladas, fabulous salsa!",OR,"Love it! Good beans, better rice, amazing enchiladas, fabulous salsa!"
3,Restaurants,4.5,Little hipster and crowded for me but yummy chocolate croissants.,OR,Little hipster and crowded for me but yummy chocolate croissants.
4,Restaurants,3.5,terrible servile. not worthy of a second visit.We decided to try this place for the,CG,terrible servile. not worthy of a second visit.We decided to try this place for the


In [ ]:
generated_df.drop(columns=["text"],axis=1,inplace=True)

In [ ]:
generated_df[500:512]

,category,rating,label,text_
500,Restaurants,3.0,OR,Very good and reliable Chinese Food in this area. Owners are very nice. If you spend over 25 dollars you get free soup or Crab Rangoon.
501,Restaurants,4.0,OR,Very neat atmosphere. Kind of like a dive bar /diner in one. \nThe food is yummy and that green jam is cool. It's something different alright.
502,Restaurants,3.0,OR,Great service from Billy. Food came out fast and enjoy the atmosphere. I ordered the garlic boneless wings and chips. Both were fantastic and would recommend.
503,Restaurants,4.0,OR,"Amazing food, great service! I've been there for brunch and dinner! Dinner time is so fun with the music and drinks! The food is sooooo flavorful!"
504,Restaurants,1.5,OR,The red velvet coffee here is terrible. Plain and simple. The only reason I give it a 2 star is because the hash browns were normal
505,Restaurants,4.0,OR,"LOVE THIS PLACE!!! \n\nAgain, I ordered the fried chicken ranch bowl. You can never go wrong with chicken, bacon, or ranch! \n\nAsk for Whitney! She's ahhhh-mazing!!!"
506,Restaurants,4.5,CG,"Oh for the love of God, you guys are amazing!!!\n\nAnd for the love of God, you guys are amazing!!!\n"
507,Restaurants,3.0,CG,No badmouthing it. It's a nice place to watch a game or catch up with a friend.
508,Restaurants,3.5,CG,Best burritos I have had in a long time. I will say that the best thing about this place is the burrito
509,Restaurants,3.5,CG,"A truly unique, eclectic restaurant that has the best food in Nashville. \n\nThe food is always delicious and the service is always"


In [ ]:
generated_df.to_csv("Generated_Sentences_Restaurant.csv", index=None)

In [ ]:
%cp /content/Generated_Sentences_Restaurant.csv /content/drive/MyDrive/YelpDataset/generatedData